In [51]:
from google.cloud import bigquery
project_id="students-group2"
client = bigquery.Client(project=project_id)

In [52]:
client.list_tables("students-group2.fatma_aziz")

In [53]:
# les noms des colonnes de la table movies 
movies = client.get_table("students-group2.fatma_aziz.movies")
print([schema_field.name for schema_field in movies.schema])

['movieId', 'title', 'genres']


In [54]:
# le nom des colonnes de la table ratings 
ratings = client.get_table("students-group2.fatma_aziz.ratings")
print([schema_field.name for schema_field in ratings.schema])

['userId', 'movieId', 'rating', 'timestamp']


In [55]:
# Dataframes 
df_movies = client.query("SELECT movieId, title, genres FROM `students-group2.fatma_aziz.movies`").to_dataframe()
df_ratings = client.query("SELECT userId, movieId, rating, timestamp FROM `students-group2.fatma_aziz.ratings`").to_dataframe()

In [56]:
df_movies

,movieId,title,genres
0,126929,Li'l Quinquin ( ),(no genres listed)
1,135460,Pablo (2012),(no genres listed)
2,138863,The Big Broadcast of 1936 (1935),(no genres listed)
3,141305,Round Trip to Heaven (1992),(no genres listed)
4,141472,The 50 Year Argument (2014),(no genres listed)
...,...,...,...
10324,85896,Tribute to a Bad Man (1956),Western
10325,103570,Dead Man's Burden (2012),Western
10326,105223,Colorado Territory (1949),Western
10327,128360,The Hateful Eight (2015),Western


In [57]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10329 entries, 0 to 10328
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  10329 non-null  Int64 
 1   title    10329 non-null  object
 2   genres   10329 non-null  object
dtypes: Int64(1), object(2)
memory usage: 252.3+ KB


In [58]:
df_ratings

,userId,movieId,rating,timestamp
0,1,204,0.5,1217895786
1,1,256,0.5,1217895764
2,1,277,0.5,1217895772
3,1,719,0.5,1217895799
4,1,45950,0.5,1217897813
...,...,...,...,...
105334,668,93040,5.0,1331051757
105335,668,98154,5.0,1353208964
105336,668,101862,5.0,1373168467
105337,668,106916,5.0,1388740601


In [59]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105339 entries, 0 to 105338
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     105339 non-null  Int64  
 1   movieId    105339 non-null  Int64  
 2   rating     105339 non-null  float64
 3   timestamp  105339 non-null  Int64  
dtypes: Int64(3), float64(1)
memory usage: 3.5 MB


In [61]:
print("le nombre de films total:",len(df_movies['movieId'].value_counts()))

le nombre de films total: 10329


In [62]:
print("le nombre de films ayant des ratings:",len(df_ratings['movieId'].value_counts()))

le nombre de films ayant des ratings: 10325


In [63]:
print("le nombre d'utilisateurs:",len(df_ratings['userId'].value_counts()))

le nombre d'utilisateurs: 668


In [64]:
print("les valeurs possibles de rating:", df_ratings['rating'].value_counts())

les valeurs possibles de rating: rating
4.0    28880
3.0    21729
5.0    14856
3.5    12237
4.5     8187
2.0     7943
2.5     5484
1.0     3258
1.5     1567
0.5     1198
Name: count, dtype: int64


In [65]:
print("les genres de films:", df_movies['genres'].value_counts())

les genres de films: genres
Drama                                               1385
Comedy                                               826
Comedy|Drama                                         465
Drama|Romance                                        421
Comedy|Romance                                       363
                                                    ... 
Adventure|Romance|War|Western                          1
Adventure|Romance|Western                              1
Adventure|Thriller|Western                             1
Action|Adventure|Thriller|War                          1
Adventure|Fantasy|Horror|Romance|Sci-Fi|Thriller       1
Name: count, Length: 938, dtype: int64
